# Mallin hienosäätäminen

In [1]:
import fastai.text as fatext
import numpy as np
import sentencepiece as sp
from typing import List

In [2]:
spm = sp.SentencePieceProcessor()
spm.Load("iso-lc.model")

True

In [3]:
class SPMTokenizer:
	def __init__(self, lang: str):
		pass
	def tokenizer(self, text: str) -> List[str]:
		return spm.EncodeAsPieces(text.lower())
	def add_special_cases(self, *args):
		pass

tokenizer = fatext.transform.Tokenizer(SPMTokenizer)

In [4]:
def loadVocab(filename: str):
	with open(filename, "r") as f:
		tokens = [l.strip().split()[0] for l in f]
	
	return fatext.transform.Vocab(tokens)

vocab = loadVocab("iso-lc.vocab")

## Datatiedosto:

In [5]:
data_file = "data/vaalidata.csv"
df = fatext.pd.read_csv(data_file)
df.head()

,"Kysymys 20, vastaus 4, perustelu: Turvapaikanhakijoiden suuri tulijamäärä vuonna 2015 on konkreettisestikin lisännyt turvattomuutta. Mukana on tullut Suomen olosuhteisiin uusiakin ilmiöitä , kuten joukolla tapahtuvaa seksuaalista ahdistelua. Maahanmuuttajaryhmiä on yliedustettuina muissakin seksuaalirikoksissa. Nykyinen turvapaikanhakujärjestelmä mahdollistaa myös terroristirikollisten liikkumisen , ja siksi siihen on tehtävä muutos. Muutos, joka estää Suomen rajoilta turvapaikanhakemisen. Tämä poistaisi monia ongelmia."
0,"Kysymys 12, vastaus 2, perustelu: En koe tunte..."
1,"Kysymys 56, vastaus 4, perustelu: Rahoitus on ..."
2,"Kysymys 52, vastaus 5, perustelu: Espoossakin ..."
3,"Kysymys 28, vastaus 5, perustelu: Äänestäjän t..."
4,"Kysymys 13, vastaus 4, perustelu: Suomalainen ..."


In [6]:
data = fatext.data.TextLMDataBunch.from_csv(".", data_file, text_cols=0, tokenizer=tokenizer, vocab=vocab)
data.show_batch()

idx,text
0,": ▁pakko ▁haa lia ▁tulevaisuudessa ▁työntekijöitä ▁sieltä ▁mistä ▁saadaan , ▁kun ▁ruotsalaiset , ▁norjalais et ▁ja ▁saksalaiset ▁eivät ▁tänne ▁kuitenkaan ▁suostu ▁tulemaan . ▁xx bo s ▁kysymys ▁9 , ▁vastaus ▁2 , ▁perustelu : ▁sama ▁sille ▁kuka ▁palvelu n ▁tuottaa , ▁kunhan ▁on ▁laadu kasta ▁ja ▁kustannus teho kasta . ▁xx bo s ▁kysymys ▁23 , ▁vastaus ▁4 , ▁perustelu : ▁viha puhe eseen ▁usein ▁liittyvä ▁muiden ▁sanan vapauden"
1,". ▁perhe et ▁kaipaa vat ▁lisää ▁jo usta vuutta ▁ja ▁valinna n vapautta . ▁ei ▁ole ▁tarkoitukse nmukais ta ▁edus kunnasta ▁käsin ▁yrittää ▁sää nne llä , ▁mikä ▁on ▁kullekin ▁perhe elle ▁parasta . ▁jokainen ▁perhe ▁lähtökohta isesti ▁haluaa ▁varmasti ▁lapsensa ▁ja ▁perheensä ▁parasta . ▁on ▁tehtävä ▁kokonais valta isesti ▁perhe ystävällis tä ▁politiikka a . ▁se ▁tarkoittaa ▁panos tuksia ▁varhais kas vat ukseen ▁ja ▁koulu tukseen , ▁paran"
2,"▁kysymys ▁30 , ▁vastaus ▁5 , ▁perustelu : ▁rake n tava an ▁yhteis työhön ▁kuuluu ▁toisten ▁huomioon ▁o ttaminen . ▁politiikka ▁ei ▁ole ▁yksilö suo ri tus ▁vaan ▁joukkue peli ä . ▁muut ▁tulee ▁ottaa ▁huomioon ▁myös ▁muilla ▁elämän alueilla . ▁xx bo s ▁kysymys ▁26 , ▁vastaus ▁2 , ▁perustelu : ▁ihmisille ▁täytyy ▁yrittää ▁saada ▁yhtä ▁hyvät ▁asema t , ▁pare mmassa ▁asemassa ▁olevat ▁voisivat ▁auttaa ▁siinä ▁xx"
3,"▁emme ▁voi ▁tehdä ▁tarvi ttavia ▁investoi n teja , ▁joilla ▁on ▁suomea ▁suurempi ▁vaikutus . ▁on ▁myös ▁tärkeää ▁huolehti a , ▁etteivät ▁toimenpitee t ▁kurji sta ▁tavallisen ▁ihmisen ▁ar kea . ▁xx bo s ▁kysymys ▁12 , ▁vastaus ▁1 , ▁perustelu : ▁alle ▁18- vuotias ▁on ▁ala ikäinen . ▁xx bo s ▁kysymys ▁6 , ▁vastaus ▁1 , ▁perustelu : ▁yleensä ▁meno jen ▁karsi minen ▁osu u ▁juuri ▁ki"
4,"▁ovat ▁erityisen ▁tärkeitä . ▁xx bo s ▁kysymys ▁10 , ▁vastaus ▁2 , ▁perustelu : ▁ulko ista misia ▁yksityis ille ▁toimi joille ▁ei ▁tule ▁lisätä ▁ennenkuin ▁saadaan ▁tiuk enne ttu ▁lainsäädäntö ▁asiasta . ▁suurten ▁toimi joiden ▁välin pi tä mätön ▁ja ▁ahne ▁ase nne ▁ vanh uksia ▁kohtaan ▁on ▁ollut ▁käsi ttä mätöntä . ▁xx bo s ▁kysymys ▁19 , ▁vastaus ▁1 , ▁perustelu : ▁korkeakoulu ja ▁on ▁mielestäni"


In [7]:
learner = fatext.learner.language_model_learner(data, fatext.models.AWD_LSTM, pretrained=False)

## Suorita vain jos haluat kouluttaa mallin:

In [8]:
learner.load("lstm2")

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (44467 items)
x: LMTextList
▁xx bo s ▁kysymys ▁22 , ▁vastaus ▁1 , ▁perustelu : ▁suomi ▁tunnetaan ▁puolueettoma na ▁maan a ▁ja ▁rauhan ▁ rakentaja na ▁maailmassa . ▁tähän ▁maine eseen ▁ei ▁ole ▁syytä ▁tehdä ▁muutoksia .,▁xx bo s ▁kysymys ▁28 , ▁vastaus ▁5 , ▁perustelu : ▁valtio vallan ▁velvollisuus ▁on ▁työskennellä ▁sen ▁kansan ▁edu n ▁eteen , ▁joka ▁sille ▁on ▁vallan ▁antanut,▁xx bo s ▁kysymys ▁21 , ▁vastaus ▁5 , ▁perustelu : ▁pakko ▁haa lia ▁tulevaisuudessa ▁työntekijöitä ▁sieltä ▁mistä ▁saadaan , ▁kun ▁ruotsalaiset , ▁norjalais et ▁ja ▁saksalaiset ▁eivät ▁tänne ▁kuitenkaan ▁suostu ▁tulemaan .,▁xx bo s ▁kysymys ▁9 , ▁vastaus ▁2 , ▁perustelu : ▁sama ▁sille ▁kuka ▁palvelu n ▁tuottaa , ▁kunhan ▁on ▁laadu kasta ▁ja ▁kustannus teho kasta .,▁xx bo s ▁kysymys ▁23 , ▁vastaus ▁4 , ▁perustelu : ▁viha puhe eseen ▁usein ▁liittyvä ▁muiden ▁sanan vapauden ▁rajoitta minen ▁tulee ▁tunnistaa ▁niin ▁lainsäädännö ssä ▁kuin ▁oikeus käy tä n n össä 

In [9]:
learner.fit_one_cycle(5)

epoch,train_loss,valid_loss,accuracy,time
0,3.920649,3.636027,0.396093,1:37:58
1,3.571706,3.336449,0.422474,1:36:25
2,3.400970,3.226527,0.433598,1:36:40
3,3.307856,3.189011,0.438441,1:36:50
4,3.279908,3.184075,0.438862,1:36:48


In [10]:
learner.save("lstm2-vaalidata")

## Suorita tästä eteenpäin valmiilla mallilla:

In [11]:
learner.load("lstm2-vaalidata")

ValueError: loaded state dict contains a parameter group that doesn't match the size of optimizer's group

In [12]:
print(learner.predict("olit", 200, temperature=0.7).replace(" ", "").replace("▁", " ").replace("xxbos", "\n"))

olit realistinen ratkaisu, ja sen vuoksi ei ole tarvetta pyrkiä järjestämään ulkomaista yhteistyötä. 
 kysymys 6, vastaus 2, perustelu: valtion menoja ei tule enää nostaa, joten menojen karsiminen ei välttämättä ole osa suomen valtion verotusta. 
 kysymys 53, vastaus 4, perustelu: se on hyvä vaihtoehto. ydinvoima on tärkeää ja on hyvä olla mukana myös tuulivoiman vastaisessa työssä. 
 kysymys 91, vastaus 2, perustelu: päärata on tampereella. lähijunaliikenne on tärkeä elinkeino ja liikennepoliittinen investointi. 
 kysymys 16, vastaus 1, perustelu: perhevapaauudistuksen tulee olla liian pidentävää. 
 kysymys 30, vastaus 5, perustelu: on oikein nähdä vaivaa, mutta ei kuitenkaan. 
 kysymys 26, vastaus 2, perustelu: koko yhteiskunta ei voi menestyä. 
 kysymys 30, vastaus 4, perustelu: jokaisella ihmisellä on oikeus hyvään elämään. 
 kysymys 9, vastaus 2, perustelu: julkis
